In [ ]:
from skimage.feature import greycomatrix, greycoprops
from sklearn.decomposition import PCA
import pandas as pd
from collections import defaultdict
import random
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import glob
import seaborn as sns

In [ ]:
test_fstrig = '/scratch/hoerl/dna_sir_ageing/data/{}/*.png'
clss = ['yng_ut_1', 'yng_ut_2', 'yng_ut_3', 'old_ut1', 'old_ut2', 'old_ut3']

dists = np.arange(0, 30, 2).astype(np.int)
#angles = np.linspace(0, np.pi/2, 10)
angles = [0, np.pi/2]
n_sample = 200
props = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']
res = defaultdict(list)

for cls in clss:
    test_files = glob.glob(test_fstrig.format(cls))
    #for test_file in test_files:
    for test_file in random.sample(test_files, n_sample):
        img = imread(test_file)
        gcm = greycomatrix(img, dists, angles, symmetric=True)
        fts = np.array([greycoprops(gcm, p) for p in props])
        res['feats'].append(fts.ravel())
        res['cls'].append(cls)

pca = PCA()
tr = pca.fit_transform(np.array(res['feats']))

res['pc0'] = tr[:,0]
res['pc1'] = tr[:,1]

#plt.plot(tr[:n_sample,0], tr[:n_sample,1], '*')
#plt.plot(tr[n_sample:,0], tr[n_sample:,1], '*')

df = pd.DataFrame.from_dict(res)

plt.figure(figsize=(10,10))
sns.scatterplot(data=df, x='pc0', y='pc1', hue='cls')